# 第8章: ニューラルネット

## 70. 単語埋め込みの読み込み

In [6]:
# 学習済み単語ベクトルのダウンロード
!pip install gdown
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM -O data/GoogleNews-vectors-negative300.bin.gz

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: lightning-lite 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of lightning-lite or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
/home/j329nish/.local/lib/python3.10/

In [3]:
!pip install numpy==1.24.3 gensim

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: lightning-lite 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of lightning-lite or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [19]:
import numpy as np
from gensim.models import KeyedVectors

# モデルを読み込む
model_path = "./data/GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

vocab_size = len(model.key_to_index) + 1
embedding_dim = model.vector_size

embedding_matrix = np.zeros((vocab_size, embedding_dim))

token2id = {"<PAD>": 0}
id2token = {0: "<PAD>"}

for idx, word in enumerate(model.key_to_index, start=1):
  embedding_matrix[idx] = model[word]
  token2id[word] = idx
  id2token[idx] = word

print(f"Embedding matrix shape: {embedding_matrix.shape}")

# 属性一覧の表示  https://aiacademy.jp/media/?p=2013
# 単語埋め込み    https://nlp100.vercel.app/docs/ch08/ch08.html

Embedding matrix shape: (3000001, 300)


## 71. データセットの読み込み

In [11]:
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip -P data/
!unzip -o data/SST-2.zip -d data/
!rm data/SST-2.zip

--2025-04-22 20:20:20--  https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
dl.fbaipublicfiles.com (dl.fbaipublicfiles.com) をDNSに問いあわせています... 54.239.168.70, 54.239.168.65, 54.239.168.51, ...
dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|54.239.168.70|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 7439277 (7.1M) [application/zip]
`data/SST-2.zip' に保存中

SST-2.zip           100%[===================>]   7.09M  32.9MB/s    in 0.2s    

2025-04-22 20:20:21 (32.9 MB/s) - `data/SST-2.zip' へ保存完了 [7439277/7439277]

Archive:  data/SST-2.zip
   creating: data/SST-2/
  inflating: data/SST-2/dev.tsv      
   creating: data/SST-2/original/
  inflating: data/SST-2/original/README.txt  
  inflating: data/SST-2/original/SOStr.txt  
  inflating: data/SST-2/original/STree.txt  
  inflating: data/SST-2/original/datasetSentences.txt  
  inflating: data/SST-2/original/datasetSplit.txt  
  inflating: data/SST-2/original/dictionary.txt  
  inflating: data/SST-2/original/original_rt_snip

In [22]:
import csv
import collections
import torch

def make_dict(file_name):
  dictionary = []
  with open(file_name, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    next(reader)
    for row in reader:
      input_ids = [token2id[word] for word in row[0].split() if word in token2id]
      if len(input_ids) == 0:
        continue
      dictionary.append({
        'text': row[0], 
        'label': torch.tensor([float(row[1])]), 
        'input_ids': torch.tensor(input_ids)
      })
  return dictionary

train_dict =  make_dict('./data/SST-2/train.tsv')
dev_dict = make_dict('./data/SST-2/dev.tsv')

print('学習データのリスト：')
for i in range(5):
  print(train_dict[i])
print('')

print('検証データのリスト：')
for i in range(5):
  print(dev_dict[i])

# torchのtensorについて https://qiita.com/mathlive/items/241bfb42d852bb801b96

学習データのリスト：
{'text': 'hide new secretions from the parental units ', 'label': tensor([0.]), 'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])}
{'text': 'contains no wit , only labored gags ', 'label': tensor([0.]), 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
{'text': 'that loves its characters and communicates something rather beautiful about human nature ', 'label': tensor([1.]), 'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,
         1964])}
{'text': 'remains utterly satisfied to remain the same throughout ', 'label': tensor([0.]), 'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}
{'text': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 'label': tensor([0.]), 'input_ids': tensor([    6,    12,  1445, 43789,    12, 10946,    76, 41349,    42])}

検証データのリスト：
{'text': "it 's a charming and often affecting journey . ", 'label': tensor([1.]), 'input

## 72. Bag of wordsモデルの構築

## 73. モデルの学習

## 74. モデルの評価

## 75. パディング

## 76. ミニバッチ学習

## 77. GPU上での学習

## 78. 単語埋め込みのファインチューニング

## 79. アーキテクチャの変更